In [ ]:
!pip install pyspark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

In [ ]:
!tar xvzf spark-3.4.1-bin-hadoop3.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

In [ ]:
!pip install findspark

In [ ]:
!echo $SPARK_HOME

In [ ]:
import findspark
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import  SparkContext
from pyspark.sql import functions as f
appName = "PySpark Task 1"
#master = "spark://10.3.100.4:7077"
master = 'local[*]'
# Create Spark session with Hive supported.
spark = SparkSession.builder \
    .enableHiveSupport() \
    .appName(appName) \
    .getOrCreate()

In [ ]:
df_emp = spark.read.json("/content/sample_data/anscombe.json")
df_emp.show(5,False)

In [ ]:
from pyspark.sql import Row

spark = SparkSession.builder.appName("OlympicDisciplines").getOrCreate()

data = [
    Row(row_id=1, discipline="Athletics", season="summer"),
    Row(row_id=2, discipline="Swimming", season="summer"),
    Row(row_id=3, discipline="Gymnastics", season="summer"),
    Row(row_id=4, discipline="Skiing", season="winter"),
    Row(row_id=5, discipline="Hockey", season="winter"),
    Row(row_id=6, discipline="Cycling", season="summer"),
    Row(row_id=7, discipline="Figure_skating", season="winter"),
    Row(row_id=8, discipline="Rowing", season="summer"),
    Row(row_id=9, discipline="Biathlon", season="winter"),
    Row(row_id=10, discipline="Basketball", season="summer"),
]

df = spark.createDataFrame(data)

df.coalesce(1).write.option("header", "true").option("delimiter", "\t").csv("/content/1olympic_disciplines.csv")

spark.stop()

In [ ]:
#проверка
spark = SparkSession.builder.appName("OlympicDisciplines").getOrCreate()
df = spark.read.option("delimiter", "\t").option("encoding", "UTF-8").option("header", "true").csv("/content/1olympic_disciplines.csv")
df.show()

+------+--------------+------+
|row_id|    discipline|season|
+------+--------------+------+
|     1|     Athletics|summer|
|     2|      Swimming|summer|
|     3|    Gymnastics|summer|
|     4|        Skiing|winter|
|     5|        Hockey|winter|
|     6|       Cycling|summer|
|     7|Figure_skating|winter|
|     8|        Rowing|summer|
|     9|      Biathlon|winter|
|    10|    Basketball|summer|
+------+--------------+------+



In [ ]:
from pyspark.sql.functions import count

spark = SparkSession.builder.appName("AthletesAnalysis").getOrCreate()

df = spark.read.option("delimiter", ";").option("encoding", "UTF-8").option("header", "true").csv("/content/Athletes.csv")

result = df.groupBy("Discipline").agg(count("*").alias("TotalAthletes"))

result.write.format("parquet").save("/content/athletes_by_discipline.parquet")

# Завершаем сессию Spark
spark.stop()

In [ ]:
#проверка
spark = SparkSession.builder.appName("AthletesAnalysis").getOrCreate()
df = spark.read.format("parquet").load("/content/athletes_by_discipline.parquet")
df.show()

+--------------------+-------------+
|          Discipline|TotalAthletes|
+--------------------+-------------+
|              Tennis|          178|
|              Boxing|          270|
|   Marathon Swimming|           49|
|                Golf|          115|
|              Rowing|          496|
|   Baseball/Softball|          220|
|                Judo|          373|
|             Sailing|          336|
|            Swimming|          743|
|Cycling BMX Frees...|           19|
|          Basketball|          280|
|            Handball|          343|
| Rhythmic Gymnastics|           95|
|              Karate|           77|
|           Triathlon|          106|
|           Badminton|          164|
|        Canoe Sprint|          236|
|           Athletics|         2068|
|       Cycling Track|          208|
|    Beach Volleyball|           90|
+--------------------+-------------+
only showing top 20 rows



In [ ]:
spark = SparkSession.builder.appName("NewAthletesAnalysis").getOrCreate()

athletes_df = spark.read.option("delimiter", ";").option("encoding", "UTF-8").option("header", "true").csv("/content/Athletes.csv")

TotalAthletes = df.groupBy("Discipline").agg(count("*").alias("TotalAthletes"))

my_athletes_df = spark.read.option("delimiter", "\t").option("encoding", "UTF-8").option("header", "true").csv("/content/1olympic_disciplines.csv")

final_result = my_athletes_df.join(TotalAthletes, my_athletes_df.discipline == TotalAthletes.Discipline, "inner").select(my_athletes_df["*"], TotalAthletes["TotalAthletes"])

final_result.write.format("parquet").save("/content/final_analytics.parquet")

spark.stop()

In [ ]:
#проверка
spark = SparkSession.builder.appName("AthletesAnalysis").getOrCreate()
df = spark.read.format("parquet").load("/content/final_analytics.parquet")
df.show()

+------+----------+------+-------------+
|row_id|discipline|season|TotalAthletes|
+------+----------+------+-------------+
|     8|    Rowing|summer|            1|
|     2|  Swimming|summer|            1|
|    10|Basketball|summer|            1|
|     1| Athletics|summer|            1|
|     5|    Hockey|winter|            1|
+------+----------+------+-------------+



In [ ]:
#проверка
spark = SparkSession.builder.appName("OlympicDisciplines").getOrCreate()
df = spark.read.option("delimiter", ";").option("encoding", "UTF-8").option("header", "true").csv("/content/Athletes.csv")
df.show()